# Parameters

In [ ]:
model_file          = "../models/europe.json"
load_series_file    = "data/loads.csv"
noise_series_file   = "data/cost_noise.csv"
nuclear_series_file = "data/nuclear_2016.csv"
border_flows_file   = "data/border_flows_2016.csv"
output_dir          = "data/TOPF_run";

In [ ]:
# year = 2020
# i    = 4

# model_file          = "../models/europe.json"
# load_series_file    = "data/loads_$(year)_$i.csv"
# noise_series_file   = "data/cost_noise_$(year)_$i.csv"
# nuclear_series_file = "data/nuclear_$year.csv"
# border_flows_file   = "data/border_flows_$year.csv"
# output_dir          = "data/TOPF_$(year)_$i"

# Network

In [ ]:
using TemperateOptimalPowerFlow
using CSV, DataFrames, DataDrop

Import the model:

In [ ]:
network = import_model(model_file)

# Assign ramp constraints

Add ramp constraint by defining a maximum ramp rate of 200 MW / hour for coal generators 

In [ ]:
for gen ∈ values(network["gen"])
    gen["max_ramp_rate"] = gen["aggregated_type"] == "coal" ? 2.0 : 0.0
end

# Nuclear production

Import actual time series for nuclear generators:

In [ ]:
nuc_series_df = CSV.read(nuclear_series_file, DataFrame)
sort!(nuc_series_df, :id)
nuc_series_df.id .= string.(nuc_series_df.id)
nuc_series_df

Separate nuclear generators as non-dispatchable:

In [ ]:
split_nondispatchable!(network, nuc_series_df.id)

Export the time series:

In [ ]:
filter!(row -> string(row.id) in keys(network["gen_nd"]), nuc_series_df);

In [ ]:
nuc_ids = nuc_series_df.id

In [ ]:
nuc_series = Matrix(nuc_series_df[:, 2:end]) / 100.

# Loads

List of all countries in the network:

In [ ]:
countries = Set([bus["country"] for bus ∈ values(network["bus"])])

### Compute expected production by country

Determine the total expected production:

In [ ]:
expected_prod = Dict(country => 0.0 for country ∈ countries)

for gen ∈ values(network["gen"])
    expected_prod[gen["country"]] += gen["pexp"]
end

sort(expected_prod, byvalue=true, rev=true)

### Compute import/export balance by country

Export balance by country:

In [ ]:
border_flows_df = CSV.read(border_flows_file, DataFrame);

In [ ]:
border_flows = Dict{String, Float64}()
for (country, flow) in eachrow(border_flows_df)
    border_flows[country] = flow / 100.0
end
border_flows["XX"] = 0.0

sort(border_flows, byvalue=true, rev=true)

Check: total out flow adds up to zero

In [ ]:
sum(values(border_flows))

### Compute nuclear production by country

Nuclear production by country:

In [ ]:
avg_nuclear = sum(nuc_series, dims=2) / size(nuc_series, 2)

nuclear_prod = Dict(country => 0.0 for country ∈ countries)
for i = 1:length(nuc_ids)
    country = network["gen_nd"][nuc_ids[i]]["country"]
    nuclear_prod[country] += avg_nuclear[i]
end
nuclear_prod

### Compute total load by country

Determine annual load by country:

In [ ]:
total_load = Dict(country => prod + nuclear_prod[country] - border_flows[country]
    for (country, prod) ∈ expected_prod)

sort(total_load, byvalue=true, rev=true)

### Import loads

In [ ]:
normalized_loads_df = CSV.read(load_series_file, DataFrame)
sort!(normalized_loads_df, :id)
normalized_loads_df.id .= string.(normalized_loads_df.id)
normalized_loads_df

Check that the IDs match:

In [ ]:
bus_ids = normalized_loads_df.id
issetequal(bus_ids, keys(network["bus"]))

In [ ]:
normalized_loads = Matrix(normalized_loads_df[:, 2:end])

Check normalization:

In [ ]:
sum(normalized_loads) / length(normalized_loads)

Normalization by country:

In [ ]:
normalized_annual_loads = Dict(country => [] for country ∈ countries)
for (id, annual_load) ∈ Dict(bus_ids .=> sum(normalized_loads, dims=2) / size(normalized_loads, 2))
    push!(normalized_annual_loads[network["bus"][id]["country"]], annual_load)
end

normalization = Dict(country => sum(annual_loads) / length(annual_loads)
    for (country, annual_loads) ∈ normalized_annual_loads)

Check that the load assignement within each country is properly normalized:

In [ ]:
load_prop_by_country = Dict(country => 0.0 for country in countries)
for bus ∈ values(network["bus"])
    load_prop_by_country[bus["country"]] += bus["load_prop"]
end

load_prop_by_country

Compute a scaling value for each country:

In [ ]:
load_scaling = [(
        network["bus"][id]["load_prop"]
        * normalization[network["bus"][id]["country"]]
        * total_load[network["bus"][id]["country"]]
        )
    for id ∈ bus_ids]

In [ ]:
loads = load_scaling .* normalized_loads

Check that the annual load matches the expectations:

In [ ]:
sum(loads) / size(loads, 2) / sum(values(total_load))

# Generation costs

In [ ]:
gen_costs = Matrix(CSV.read(noise_series_file, DataFrame))

# Setup

In [ ]:
setup(output_dir, network, loads, gen_costs, nuc_series)

In [ ]:
readdir(output_dir)

# List bus names associated with generators

This can be used to analyse the output:

In [ ]:
gen_names = [network["bus"][string(network["gen"][id]["gen_bus"])]["name"] for id in get_ordered_gen_ids(network)];